In [1]:
import pandas as pd
import rasa
import os

In [2]:
BASE_PATH = "."

In [3]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:  
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_PATH = "/content/drive/MyDrive/NLP/CA6"

In [43]:
TRAIN_PATH = os.path.join(BASE_PATH, "Data/MCI_Internet_TrainData.xlsx")
TEST_PATH = os.path.join(BASE_PATH, "Data/MCI_Internet_TestData.xlsx")
RASA_PATH = os.path.join(BASE_PATH, "rasa")
RASA_DATA_PATH = os.path.join(RASA_PATH, "data")

In [5]:
# Read xlsx file into a pandas dataframe
train_df = pd.read_excel(TRAIN_PATH)
test_df = pd.read_excel(TEST_PATH)


In [6]:
train_df.head()

,شناسه,پاسخ,سطح ۱,سطح ۲,سطح ۳,سطح ۴,سطح ۵,پرسش ۱,پرسش ۲,پرسش ۳,...,پرسش ۶,پرسش ۷,پرسش ۸,پرسش ۹,پرسش ۱۰,پرسش ۱۱,پرسش ۱۲,پرسش ۱۳,پرسش ۱۴,پرسش ۱۵
0,1,مشترک گرامی جهت فعالسازی سرویس اینترنت (GPRS) ...,اینترنت,مشکلات,فعال سازی سرویس,NaN,NaN,برای فعال سازی سرویس gprs با استفاده از کد دس...,برای فعالسازی سرویس gprs چه شماره ای باید بگیرم؟,برای فعالسازی سرویس gprs چه کدی رو باید بزنم؟,...,چه کدی برای فعالسازی سرویس اینترنت gprs باید ...,gprs رو با استفاده از کد دستوری چجوری فعال کنم؟,سرویس اینترنت فعال سازیش با استفاده از کد دستو...,کد فعالسازی سرویس اینترنت چیه؟,چیکار باید بکنم تا سرویس اینترنتم با استفاده ا...,میخوام سرویس اینترنت gprs فعال بشه با استفاده ...,سرویس اینترنتم چی جوری با استفاده از کد دستوری...,واسه فعال شدن سرویس اینترنتم با کد دستوری چیکا...,برای فعال شدن سرویس اینترنت چه کدی رو باید بگیرم؟,برای فعال شدن اینترنت gprs با استفاده از کد دس...
1,2,مشترک گرامی جهت فعالسازی سرویس اینترنت (GPRS) ...,اینترنت,مشکلات,فعال سازی سرویس,NaN,NaN,برای فعالسازی سرویس gprs از طریق پیامک چیکار کنم؟,برای فعال سازی سرویس gprs به چه شماره ای باید ...,برای فعالسازی سرویس gprs به چه شماره ای باید ا...,...,به چه شماره ای برای فعالسازی سرویس اینترنت gpr...,gprs رو از طریق پیامک چجوری فعال کنم؟,سرویس اینترنت فعال سازیش از طریق پیامک Sms چجو...,sms فعال سازی سرویس اینترنت چیه؟,به چه شماره ای پیامک اس ام اس sms بدم تا سرویس...,میخوام سرویس اینترنت gprs فعال بشه از طریق پیا...,سرویس اینترنتم از طریق پیامک چیجوری فعال میشه؟,واسه فعال شدن سرویس اینترنتم از طریق پیامک اس ...,برای فعال شدن سرویس اینترنت چه اس ام اسی باید ...,برای فعال شدن اینترنت gprs از طریق پیامک کاری...
2,3,مشترک گرامی جهت غیر فعالسازی سرویس اینترنت (G...,اینترنت,مشکلات,غیر فعال سازی سرویس,NaN,NaN,برای غیر فعالسازی سرویس gprs از طریق کد دستوری...,برای غیر فعالسازی سرویس gprs چه شماره ای باید ...,برای غیر فعالسازی سرویس gprs چه کدی رو باید بزنم؟,...,چه کدی برای غیر فعالسازی سرویس اینترنت gprs با...,gprs رو از طریق کد دستوری چجوری غیر فعال کنم؟,سرویس اینترنت غیر فعالسازیش از طریق کد دستوری ...,کد غیر فعالسازی سرویس اینترنت چیه؟,چیکار باید بکنم تا سرویس اینترنتم از طریق کد د...,میخوام سرویس اینترنت gprs غیر فعال بشه از طریق...,سرویس اینترنتم چیجوری از طریق کد دستوری غیر فع...,واسه غیر فعال شدن سرویس اینترنتم از طریق کد دس...,برای غیر فعال شدن سرویس اینترنت چه کدی رو باید...,برای غیر فعال شدن اینترنت gprs از طریق کد دستو...
3,4,مشترک گرامی جهت غیر فعالسازی سرویس اینترنت (GP...,اینترنت,مشکلات,غیر فعال سازی سرویس,NaN,NaN,برای غیر فعال سازی سرویس gprs از طریق پیامک چ...,برای غیر فعالسازی سرویس gprs به چه شماره ای با...,برای غیر فعالسازی سرویس gprs به چه شماره ای با...,...,به چه شماره ای برای غیر فعالسازی سرویس اینترنت...,gprs رو چجوری از طریق پیامک غیر فعال کنم؟,سرویس اینترنت غیر فعالسازیش از طریق پیامک SMS‌...,sms غیر فعال سازی سرویس اینترنت چیه؟,چیکار باید بکنم تا سرویس اینترنتم از طریق پیام...,میخوام سرویس اینترنت gprs غیر فعال بشه باید چه...,سرویس اینترنتم چیجوری از طریق ارسال پیامک SMS‌...,واسه غیر فعال شدن سرویس اینترنتم از طریق پیامک...,برای غیر فعال شدن سرویس اینترنت چه اس ام اسی ب...,برای غیر فعال شدن اینترنت gprs از طریق پیامک ...
4,5,بعد از اقدام و دریافت پیامک تایید فعالسازی سرو...,اینترنت,مشکلات,فعال سازی سرویس,NaN,NaN,بعد از دریافت پیامک تایید فعالسازی اینترنت بای...,بعد از دریافت پیامک تایید فعالسازی اینترنت چیک...,پیامک فعالسازی اینترنت دریافت کردم ولی هنوز ای...,...,بعد آمدن پیامک تایید فعالسازی سرویس اینترنت چر...,بعد آمدن پیامک تایید فعالسازی سرویس اینترنت هن...,باید بعد از دریافت پیامک فعالسازی اینترنت کاری...,پیامک تایید فعالسازی سرویس اینترنم اومده ولی ه...,آیا بعد از دریافت پیامک تایید فعالسازی اینترنت...,پیامک تایید فعالسازی اینترنتم اومده و هنوز نت...,پیامکی مبنی بر فعالسازی اینترنت اومد بعدش باید...,پیامکی مبنی بر فعالسازی اینترنت اومده حالا چیک...,اس ام اس تایید فعال سازی اینترنت اومد ولی نت ن...,اس ام اس اومد فعال سازی سرویس اینترنت شما تایی...


In [7]:
deleted_cols = ["سطح ۴","سطح ۵"]
train_df.drop(deleted_cols, axis=1, inplace=True)


In [8]:
train_df_english = train_df.copy()
# Translate columns from index 2 to 4 to english
translation_dict = {
    "اینترنت": "Internet",
    "مشکلات": "Problems",
    "بسته": "Package",
    "فعال سازی سرویس": "Service Activation",
    "غیر فعال سازی سرویس": "Service Deactivation",
    "سرویس آموزش": "Educational Service",
    "مشکلات اتصال به اینترنت": "Internet Connection Problems",
    "4G و 4.5G": "4G and 4.5G",
    "خرید بسته از شاپ (فروشگاه)": "Package Purchase from Shop",
    "خرید بسته از *100#": "Package Purchase from *100#",
    "نام تجاری نوترینو": "Notrino Brand",
    "بسته‌های اینترنت پاک": "Pocket Internet Packages",
    "بسته‌های ساعات مشخص": "Specific Hour Packages",
    "بسته ویژه مشترکین جدید": "Special Package for New Subscribers",
    "استعلام بسته": "Package Inquiry",
    "لغو تمدید بسته": "Package Renewal Cancellation",
    "بسته‌های نامحدود شبانه": "Unlimited Night Packages",
    "بسته‌های اینترنت": "Internet Packages",
}
# Search only  in the columns from index 2 to 4 and replace the persian words with english words
train_df_english.iloc[:, 2:5] = train_df_english.iloc[:, 2:5].replace(translation_dict)

In [9]:
train_df_english.head(5)

,شناسه,پاسخ,سطح ۱,سطح ۲,سطح ۳,پرسش ۱,پرسش ۲,پرسش ۳,پرسش ۴,پرسش ۵,پرسش ۶,پرسش ۷,پرسش ۸,پرسش ۹,پرسش ۱۰,پرسش ۱۱,پرسش ۱۲,پرسش ۱۳,پرسش ۱۴,پرسش ۱۵
0,1,مشترک گرامی جهت فعالسازی سرویس اینترنت (GPRS) ...,Internet,Problems,Service Activation,برای فعال سازی سرویس gprs با استفاده از کد دس...,برای فعالسازی سرویس gprs چه شماره ای باید بگیرم؟,برای فعالسازی سرویس gprs چه کدی رو باید بزنم؟,چجوری سرویس اینترنت gprs با استفاده از کد دستو...,چه شماره ای برای فعالسازی سرویس اینترنت gprs ب...,چه کدی برای فعالسازی سرویس اینترنت gprs باید ...,gprs رو با استفاده از کد دستوری چجوری فعال کنم؟,سرویس اینترنت فعال سازیش با استفاده از کد دستو...,کد فعالسازی سرویس اینترنت چیه؟,چیکار باید بکنم تا سرویس اینترنتم با استفاده ا...,میخوام سرویس اینترنت gprs فعال بشه با استفاده ...,سرویس اینترنتم چی جوری با استفاده از کد دستوری...,واسه فعال شدن سرویس اینترنتم با کد دستوری چیکا...,برای فعال شدن سرویس اینترنت چه کدی رو باید بگیرم؟,برای فعال شدن اینترنت gprs با استفاده از کد دس...
1,2,مشترک گرامی جهت فعالسازی سرویس اینترنت (GPRS) ...,Internet,Problems,Service Activation,برای فعالسازی سرویس gprs از طریق پیامک چیکار کنم؟,برای فعال سازی سرویس gprs به چه شماره ای باید ...,برای فعالسازی سرویس gprs به چه شماره ای باید ا...,چجوری سرویس اینترنت gprs از طریق پیامک اس ام ا...,به کجا برای فعال سازی سرویس اینترنت gprs باید ...,به چه شماره ای برای فعالسازی سرویس اینترنت gpr...,gprs رو از طریق پیامک چجوری فعال کنم؟,سرویس اینترنت فعال سازیش از طریق پیامک Sms چجو...,sms فعال سازی سرویس اینترنت چیه؟,به چه شماره ای پیامک اس ام اس sms بدم تا سرویس...,میخوام سرویس اینترنت gprs فعال بشه از طریق پیا...,سرویس اینترنتم از طریق پیامک چیجوری فعال میشه؟,واسه فعال شدن سرویس اینترنتم از طریق پیامک اس ...,برای فعال شدن سرویس اینترنت چه اس ام اسی باید ...,برای فعال شدن اینترنت gprs از طریق پیامک کاری...
2,3,مشترک گرامی جهت غیر فعالسازی سرویس اینترنت (G...,Internet,Problems,Service Deactivation,برای غیر فعالسازی سرویس gprs از طریق کد دستوری...,برای غیر فعالسازی سرویس gprs چه شماره ای باید ...,برای غیر فعالسازی سرویس gprs چه کدی رو باید بزنم؟,چجوری سرویس اینترنت gprs رو از طریق کد دستوری ...,چه شماره ای برای غیر فعالسازی سرویس اینترنت gp...,چه کدی برای غیر فعالسازی سرویس اینترنت gprs با...,gprs رو از طریق کد دستوری چجوری غیر فعال کنم؟,سرویس اینترنت غیر فعالسازیش از طریق کد دستوری ...,کد غیر فعالسازی سرویس اینترنت چیه؟,چیکار باید بکنم تا سرویس اینترنتم از طریق کد د...,میخوام سرویس اینترنت gprs غیر فعال بشه از طریق...,سرویس اینترنتم چیجوری از طریق کد دستوری غیر فع...,واسه غیر فعال شدن سرویس اینترنتم از طریق کد دس...,برای غیر فعال شدن سرویس اینترنت چه کدی رو باید...,برای غیر فعال شدن اینترنت gprs از طریق کد دستو...
3,4,مشترک گرامی جهت غیر فعالسازی سرویس اینترنت (GP...,Internet,Problems,Service Deactivation,برای غیر فعال سازی سرویس gprs از طریق پیامک چ...,برای غیر فعالسازی سرویس gprs به چه شماره ای با...,برای غیر فعالسازی سرویس gprs به چه شماره ای با...,چجوری سرویس اینترنت gprs از طریق پیامک SMS‌ رو...,به کجا برای غیر فعالسازی سرویس اینترنت gprs با...,به چه شماره ای برای غیر فعالسازی سرویس اینترنت...,gprs رو چجوری از طریق پیامک غیر فعال کنم؟,سرویس اینترنت غیر فعالسازیش از طریق پیامک SMS‌...,sms غیر فعال سازی سرویس اینترنت چیه؟,چیکار باید بکنم تا سرویس اینترنتم از طریق پیام...,میخوام سرویس اینترنت gprs غیر فعال بشه باید چه...,سرویس اینترنتم چیجوری از طریق ارسال پیامک SMS‌...,واسه غیر فعال شدن سرویس اینترنتم از طریق پیامک...,برای غیر فعال شدن سرویس اینترنت چه اس ام اسی ب...,برای غیر فعال شدن اینترنت gprs از طریق پیامک ...
4,5,بعد از اقدام و دریافت پیامک تایید فعالسازی سرو...,Internet,Problems,Service Activation,بعد از دریافت پیامک تایید فعالسازی اینترنت بای...,بعد از دریافت پیامک تایید فعالسازی اینترنت چیک...,پیامک فعالسازی اینترنت دریافت کردم ولی هنوز ای...,پیامک فعال سازی اومده ولی هنوز نتم وصل نشده، م...,بعد از آمدن پیامک تایید فعالسازی سرویس اینترنت...,بعد آمدن پیامک تایید فعالسازی سرویس اینترنت چر...,بعد آمدن پیامک تایید فعالسازی سرویس اینترنت هن...,باید بعد از دریافت پیامک فعالسازی اینترنت کاری...,پیامک تایید فعالسازی سرویس اینترنم اومده ولی ه...,آیا بعد از دریافت پیامک تایید فعالسازی اینترنت...,پیام

In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   شناسه    50 non-null     int64 
 1   پاسخ     50 non-null     object
 2   سطح ۱    50 non-null     object
 3   سطح ۲    50 non-null     object
 4   سطح ۳    50 non-null     object
 5   پرسش ۱   50 non-null     object
 6   پرسش ۲   50 non-null     object
 7   پرسش ۳   50 non-null     object
 8   پرسش ۴   50 non-null     object
 9   پرسش ۵   50 non-null     object
 10  پرسش ۶   50 non-null     object
 11  پرسش ۷   50 non-null     object
 12  پرسش ۸   50 non-null     object
 13  پرسش ۹   50 non-null     object
 14  پرسش ۱۰  50 non-null     object
 15  پرسش ۱۱  50 non-null     object
 16  پرسش ۱۲  50 non-null     object
 17  پرسش ۱۳  50 non-null     object
 18  پرسش ۱۴  50 non-null     object
 19  پرسش ۱۵  50 non-null     object
dtypes: int64(1), object(19)
memory usage: 7.9+ KB


In [11]:
train_df = train_df_english.copy()

In [12]:
intent_cols = ["سطح ۱", "سطح ۲" , "سطح ۳"]
intents = []
intent_questions = {}

# For Each row in train_df, concatenate the columns of intent_cols, and append them intents list. then scan the intents list and if you see any duplicates, prefix them with numbers

for index, row in train_df.iterrows():
    intent = ""
    for col in intent_cols:
        intent += f"{row[col]}_"
    intents.append(intent)

In [13]:
# Break intents list into the sub-lists of duplicate items, for example [x,y,y,x,z] will become [x,x] , [y,y] , [z]
intent_groups = {}
for intent in intents:
    if intent in intent_groups.keys():
        intent_groups[intent].append(f"{intent}_{len(intent_groups[intent])+1}")
    else:
        intent_groups[intent] = [f"{intent}_1"]
            

In [14]:
intent_groups

{'Internet_Problems_Service Activation_': ['Internet_Problems_Service Activation__1',
  'Internet_Problems_Service Activation__2',
  'Internet_Problems_Service Activation__3'],
 'Internet_Problems_Service Deactivation_': ['Internet_Problems_Service Deactivation__1',
  'Internet_Problems_Service Deactivation__2'],
 'Internet_Problems_Educational Service_': ['Internet_Problems_Educational Service__1',
  'Internet_Problems_Educational Service__2'],
 'Internet_Problems_Internet Connection Problems_': ['Internet_Problems_Internet Connection Problems__1',
  'Internet_Problems_Internet Connection Problems__2',
  'Internet_Problems_Internet Connection Problems__3',
  'Internet_Problems_Internet Connection Problems__4',
  'Internet_Problems_Internet Connection Problems__5',
  'Internet_Problems_Internet Connection Problems__6'],
 'Internet_Problems_4G and 4.5G_': ['Internet_Problems_4G and 4.5G__1',
  'Internet_Problems_4G and 4.5G__2',
  'Internet_Problems_4G and 4.5G__3',
  'Internet_Problems

In [15]:
# write intent_groups into intent_group.txt
with open(os.path.join(BASE_PATH, "intent_groups.txt"), "w") as f:
    for intent_group in intent_groups.values():
        for intent in intent_group:
            f.write(f"{intent}\t")
        f.write("\n")

In [16]:
intent_group_count = {}
for intent_group, list in intent_groups.items():
    intent_group_count[intent_group] = len(list)
    

In [17]:
intent_group_count

{'Internet_Problems_Service Activation_': 3,
 'Internet_Problems_Service Deactivation_': 2,
 'Internet_Problems_Educational Service_': 2,
 'Internet_Problems_Internet Connection Problems_': 6,
 'Internet_Problems_4G and 4.5G_': 4,
 'Internet_Package_Internet Packages_': 3,
 'Internet_Package_Package Purchase from Shop_': 2,
 'Internet_Package_Package Purchase from *100#_': 1,
 'Internet_Package_Notrino Brand_': 2,
 'Internet_Package_Pocket Internet Packages_': 1,
 'Internet_Package_Specific Hour Packages_': 5,
 'Internet_Package_Special Package for New Subscribers_': 3,
 'Internet_Package_Package Inquiry_': 4,
 'Internet_Package_Package Renewal Cancellation_': 7,
 'Internet_Package_Unlimited Night Packages_': 5}

In [18]:
# for each row of train_df, make a column called intent_main which is intent += f"{row[col]}_" for all col in intent_cols
train_df["intent_main"] = ""
for index, row in train_df.iterrows():
    intent = ""
    for col in intent_cols:
        intent += f"{row[col]}_"
    train_df.loc[index, "intent_main"] = intent


In [19]:
train_df.head(1)

,شناسه,پاسخ,سطح ۱,سطح ۲,سطح ۳,پرسش ۱,پرسش ۲,پرسش ۳,پرسش ۴,پرسش ۵,...,پرسش ۷,پرسش ۸,پرسش ۹,پرسش ۱۰,پرسش ۱۱,پرسش ۱۲,پرسش ۱۳,پرسش ۱۴,پرسش ۱۵,intent_main
0,1,مشترک گرامی جهت فعالسازی سرویس اینترنت (GPRS) ...,Internet,Problems,Service Activation,برای فعال سازی سرویس gprs با استفاده از کد دس...,برای فعالسازی سرویس gprs چه شماره ای باید بگیرم؟,برای فعالسازی سرویس gprs چه کدی رو باید بزنم؟,چجوری سرویس اینترنت gprs با استفاده از کد دستو...,چه شماره ای برای فعالسازی سرویس اینترنت gprs ب...,...,gprs رو با استفاده از کد دستوری چجوری فعال کنم؟,سرویس اینترنت فعال سازیش با استفاده از کد دستو...,کد فعالسازی سرویس اینترنت چیه؟,چیکار باید بکنم تا سرویس اینترنتم با استفاده ا...,میخوام سرویس اینترنت gprs فعال بشه با استفاده ...,سرویس اینترنتم چی جوری با استفاده از کد دستوری...,واسه فعال شدن سرویس اینترنتم با کد دستوری چیکا...,برای فعال شدن سرویس اینترنت چه کدی رو باید بگیرم؟,برای فعال شدن اینترنت gprs با استفاده از کد دس...,Internet_Problems_Service Activation_


In [20]:
# Count the accumulated number of items in all values of intent_groups dictionary
intent_count = 0
for key in intent_groups.keys():
    intent_count += len(intent_groups[key])

intent_count

50

In [21]:
# Replace all spaces in columns from index 2 to 4 with underscore
train_df.iloc[:, 2:5] = train_df.iloc[:, 2:5].replace(" ", "_", regex=True)

In [22]:
train_df["intent_postfixed"] = ""

In [31]:
intent_count_tmp = {}
for index,row in train_df.iterrows():
    intent = row["intent_main"]
    intent_postfixed = ""
    if intent in intent_groups.keys():
        if intent not in intent_count_tmp.keys():
            intent_count_tmp[intent] = 0
            intent_postfixed = intent_groups[intent][0]
        else:
            intent_count_tmp[intent] += 1
            intent_postfixed = intent_groups[intent][intent_count_tmp[intent]]
    train_df.loc[index, "intent_postfixed"] = intent_postfixed

# make the intent_postfixed column lowercased
train_df["intent_postfixed"] = train_df["intent_postfixed"].str.lower()
train_df["intent_postfixed"] = train_df["intent_postfixed"].str.replace(" ", "_", regex=True)

In [33]:
train_df.head(10)

,شناسه,پاسخ,سطح ۱,سطح ۲,سطح ۳,پرسش ۱,پرسش ۲,پرسش ۳,پرسش ۴,پرسش ۵,...,پرسش ۸,پرسش ۹,پرسش ۱۰,پرسش ۱۱,پرسش ۱۲,پرسش ۱۳,پرسش ۱۴,پرسش ۱۵,intent_main,intent_postfixed
17,18,مشترک گرامی برای فعالسازی بسته مذکور 9 درصد ما...,Internet,Package,Internet_Packages,آیا بسته های اینترنت مالیات دارد؟,به بسته اینترنت هم مالیات میخوره؟,مالیات بسته های اینترنت چقدره؟,بسته اینترنت خریدم، چرا بیشتر از حسابم کم شده؟,خرید بسته شامل مالیات هم میشه؟,...,چقدر مالیات به مبلغ خرید بسته های اینترنت اضاف...,مالیات فعال سازی بسته های اینترنت چقدر است؟,به مبلغ بسته ها چیزی هم اضافه میشود؟,آیا برای فعال سازی بسته های اینترنت مالیات هم ...,چه مقدار مالیات به قیمت بسته های اینترنت هنگام...,هزینه اضافه ای به مبلغ بسته ها تغلق می گیرد؟,مالیات بسته ها همان ۹ درصد است؟,چه مقدار مالیات برای فعال سازی بسته ها در نظر ...,Internet_Package_Internet Packages_,internet_package_internet_packages__1
18,19,مشترک گرامی امکان تغییر یا لغو بسته بعد از فع...,Internet,Package,Internet_Packages,آیا میشه بعد از اینکه بسته فعال شد لغوش کنم؟,از بسته اینترنتم ناراضیم میشه لغوش کنم؟,میشه بسته اینترنتم رو تغییر بدم؟,چجوری بسته اینترنتم رو لغو کنم؟,چجوری بسته اینترنت فعلیم رو تغییر بدم؟,...,چه کدی بسته اینترنتم رو لغو میکنه؟,برای لغو بسته اینترنتم باید چیکار کنم؟,برای تغییر بسته اینترنت باید چیکار کنم؟,آیا میشه بسته اینترنتمو لغو کنم؟,آیا بسته اینترنت قابل تغییر هست؟,میشه بسته اینترنت رو عوض کرد؟,میتونم بسته اینترنت دیگه ای رو جایگزین بسته فع...,میتونم بسته اینترنت فعلیم رو لغو کنم؟,Internet_Package_Internet Packages_,internet_package_internet_packages__2
19,20,در صورتی که هزینه بسته رزرو شده از کارت بانکی...,Internet,Package,Internet_Packages,امکان لغو بسته رزرو شده پس از کسر هزینه از حسا...,در چه صورتی می توان بسته رزرو شده را پس از کسر...,در چه شرایطی امکان لغو بسته رزرو شده پس از کسر...,شرایط لازم جهت لغو بسته رزرو شده پس از کسر هزی...,می توان بسته رزرو شده را پس از کسر هزینه از کا...,...,با داشتن چه شرایطی می توان بسته رزرو شده را لغ...,میشه بسته ای که رزرو کردم رو پس از کسر هزینه ا...,میخوام بسته ای که رزرو کردم رو پس از فعال شدنش...,چگونه میتوان بسته رزرو را پس از کسر هزینه لغو ...,مبلغ بسته رزرو از حسابم کم شده ولی میخوام لغوش...,حواسم نبود رزروم فعال شد، میشه لغوش کنید؟,میخوام بسته رزروم که فعال شده لغو بشه,راه غیر فعال کردن بسته رزروی که فعال شده چیه؟,Internet_Package_Internet Packages_,internet_package_internet_packages__3
23,24,نوترینو نام تجاری و ثبت شده ی سرویس اینترنت نس...,Internet,Package,Notrino_Brand,نوترینو چیست؟,میشه درباره نوترینو توضیح بدین؟,مزایا نوترینو چیست؟,لطفن راجب نوترینو توضیح بدید,نوترینو نام تجاری چه سرویسی است؟,...,نوترینو رو میشه تعریف کنین؟,نوترینو چه تکنولوژیه که این قدر سریعه؟,میشه نوترینو رو بگین چیه و فرقش با 4 و 4.5G چیه؟,نام تجاری نوترینو چیست؟,لطفن تعریف نوترینو را بفرمایید,سرعت نوترینو با بقیه فرق داره؟,نوترینو به چی می‌گن؟,میشه نوترینو رو تعریف کنید؟,Internet_Package_Notrino Brand_,internet_package_notrino_brand__1
24,25,سرویس نوترینو به معنای همان سرویس اینترنت و بس...,Internet,Package,Notrino_Brand,فرق سرویس و بسته نوترینو چیه؟,منظور از سرویس و بسته نوترینو چیست؟,میشه تفاوت بسته و سرویس نوترینو رو بگین؟,تفاوت بسته و سرویس نوترینو با سرویس و بسته این...,سرویس نوترینو همون بسته نوترینو هست؟,...,تفاوت سرویس و بسته نوترینو با سرویس اینترنت چیه؟,سرویس و بسته نوترینو یعنی چی؟,فرق نوترینو با سرویس اینترنت چیه؟,تفاوت سرویس نوترینو با سرویس اینترنت چیه؟,بسته نوترینو با بسته اینترنت چه تفاوتی داره؟,ممکن است تفاوت بسته نوترینو با بسته اینترنت را...,تفاوت بسته نوترینو با بسته اینترنت چیه؟,فرق سرویس اینترنت و نوترینو چیست؟,Internet_Package_Notrino Brand_,internet_package_notrino_brand__2
34,35,مشترک گرامی برای استعلام بسته های اینترنت کد د...,Internet,Package,Package_Inquiry,برای استعلام بسته های اینترنت با استفاده از کد...,نحوه استعلام بسته های اینترنت با استفاده از کد...,چگونه می توان بسته های اینترنت را با استفاده ...,کد استعلام بسته های اینترنت چیست؟,برای استعلام بسته های اینترنت چه شماره ای رو ب...,...,روش استعلام بسته های اینترنت با استفاده از کد ...,از چه طریقی میشه بسته های اینترنت با استفاده ا...,بسته های اینترنت رو با استفاده از کد دستوری از...,

In [34]:
# Write Every item in intens into a txt file 
for intent in intents:
    with open(os.path.join(BASE_PATH, "intents.txt"), "a") as f:
        f.write(intent + "\n")

In [35]:
# Sort the dataframe by intent_postfixed column
train_df.sort_values(by=["intent_postfixed"], inplace=True)


In [61]:
# Replace all line breaks in second column of train_df with triple space
train_df.iloc[:, 1] = train_df.iloc[:, 1].str.replace("\n", "   ", regex=True)


In [52]:
# Delete nlu.yml file if exists
# Write intents and examples in rasa.yml format, intent in each row is the intent_postfixed column of train_df and examples are columns from index 2 to 19

if os.path.exists(os.path.join(RASA_DATA_PATH, "nlu.yml")):
    os.remove(os.path.join(RASA_DATA_PATH, "nlu.yml"))

with open(os.path.join(RASA_DATA_PATH, "nlu.yml"), "a") as f:
    f.write("version: \"3.1\"\n\n")
    f.write(f"nlu: \n")        
    for index, row in train_df.iterrows():
        f.write(f"-  intent:  faq/{row['intent_postfixed']}\n")
        f.write(f"   examples:  |\n")
        for i in range(5, 20):
            if not pd.isna(row[i]):
                f.write(f"    - {row[i]}\n")

In [64]:
# Delete responses.yml file if exists
# Write responses in rasa.yml format, intent in each row is the intent_postfixed column of response is the column with the index of 1

if os.path.exists(os.path.join(RASA_PATH, "domain.yml")):
    os.remove(os.path.join(RASA_PATH, "domain.yml"))

with open(os.path.join(RASA_PATH, "domain.yml"), "a") as f:
    f.write("version: \"3.1\"\n\n")
    f.write("intents:\n")
    f.write("  - faq\n\n")
    f.write(f"responses: \n")        
    for index, row in train_df.iterrows():
        f.write(f"  utter_faq/{row['intent_postfixed']}:\n")
        f.write(f"  - text:  |\n")
        f.write(f"      {row[1]}\n")
    f.write("session_config:\n")
    f.write("  session_expiration_time: 60\n")
    f.write("  carry_over_slots_to_new_session: true\n")